In [271]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from math import sqrt
import random
from collections import deque
import itertools
from IPython.display import display_html, Image

import pickle
import pydotplus
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.externals.six import StringIO  
from sklearn.tree import export_graphviz
from sklearn import preprocessing

from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, Span, Label
from bokeh.palettes import Category10

pd.set_option('display.float_format', lambda x: '%.5f' % x)

pd.set_option("display.max_rows", 8)
pd.set_option("display.max_columns", 20)

def frange(start, stop, step):
    i = start
    while i < stop:
        yield round(i, ndigits=2)
        i += step
        
def save_model(model, filepath):
    with open(filepath, 'wb') as file:
        pickle.dump(model, file)
        
def load_model(filepath):
    with open(filepath, 'rb') as file:
        return pickle.load(file)

# Data Preperation

In [372]:
cases_US = pd.read_csv("../data/US_all_vars.csv").iloc[:,1:]
cases_US['Date'] = pd.to_datetime(cases_US['Date'], cache=True)
# cases_US['Date'] = cases_US['Date'].apply(lambda x:x.toordinal())
cases_US = cases_US.drop(['FIPS', 'Country_Region', 'Total_Cases', 'State', 'County_FIPS',
                          'Phase.0', 'Phase.1', 'Phase.2', 'Phase.3', 'Abbreviation', 'Month',
                          'Lat', 'Long'],axis=1)

# cases_US = cases_US.groupby(['County', 'Date']).agg({
#     'Restriction Rating': pd.Series.mode,
#     'Governer.Party': pd.Series.mode,
#     'Current_Phase': pd.Series.mode,
#     'Cases_Delta': np.sum,
#     'Avg_Temp': np.mean,
#     'Protest_Count': np.sum,
#     'Perc.Over.65': np.mean,
#     'Perc.White': np.mean,
#     'Perc.Female': np.mean,
#     'Perc.Black': np.mean,
#     'Perc.Native': np.mean,
#     'Perc.Asian': np.mean,
#     'Perc.Pac.Island': np.mean,
#     'Perc.Mixed': np.mean,
#     'Perc.His.Lat': np.mean,
#     'Perc.Foreign.Born': np.mean,
#     'Avg.Person.Per.Household': np.mean,
#     'POP.2019': np.sum,
#     'Area.sq.km': np.sum,
#     'PopDensity': np.mean,
#     'Cases_2W': np.sum
# }).reset_index().dropna()

cases_US = cases_US.drop(['Area.sq.km', 'POP.2019'], axis=1)
cases_US_curr = cases_US[cases_US['Date'] == pd.Timestamp('2020-06-16')]

cases_US = cases_US.set_index(['County', 'Date'])

In [373]:
cases_US_curr = (pd.concat([cases_US_curr, pd.get_dummies(cases_US_curr['Restriction Rating']),
          pd.get_dummies(cases_US_curr['Governer.Party'])], axis=1)
 .drop(['Current_Phase', 'Restriction Rating', 'Governer.Party', 'Date'], axis=1)).set_index('County')
temp = cases_US_curr['Cases_2W']
cases_US_curr = cases_US_curr.drop('Cases_2W', axis=1)
cases_US_curr['Cases_2W'] = temp
cases_US_curr = cases_US_curr.dropna()

In [374]:
def calc_vif(X, thresh=10.0):
    # Calculating VIF
    X_numeric = X.select_dtypes(['float64', 'int64'])
    vif = pd.DataFrame()
    variables = list(range(X_numeric.shape[1]))
    dropped = pd.DataFrame(columns=['variable','VIF'])
    
    while True:
        vif = pd.DataFrame()
        vif["variable"] = X_numeric.iloc[:, variables].columns
        vif["VIF"] = [variance_inflation_factor(X_numeric.iloc[:, variables].values, i) 
                  for i in range(X_numeric.iloc[:, variables].shape[1])]
        max_i = vif['VIF'].idxmax()
        if(vif['VIF'].max() > thresh):
            # Keep avg temp as a variable and remove 2nd largest vif value
            if X_numeric.iloc[:, variables].columns[max_i] == 'Avg_Temp':
                max_2nd = vif['VIF'].nlargest(2).iloc[1:].values[0]
                if max_2nd <= thresh:
                    continue
                else:
                    max_i = vif['VIF'].nlargest(2).iloc[1:].index[0]
            dropped = dropped.append({
                'variable': X_numeric.iloc[:, variables].columns[max_i],
                'VIF': vif.loc[max_i, "VIF"]
            }, ignore_index=True)
            variables.pop(max_i)
            continue
        break

    return vif, dropped

vif, dropped = calc_vif(cases_US_curr.iloc[:,:-9])

In [369]:
cases_US_curr.iloc[:,:-9]

,Cases_Delta,Avg_Temp,Protest_Count,Perc.Over.65,Perc.White,Perc.Female,Perc.Black,Perc.Native,Perc.Asian,Perc.Pac.Island,Perc.Mixed,Perc.His.Lat,Perc.Foreign.Born,Avg.Person.Per.Household,PopDensity
County,,,,,,,,,,,,,,,
Abbeville,0.00000,76.10000,0.00000,18.70000,69.90000,51.40000,28.20000,0.30000,0.40000,0.00000,1.30000,1.20000,1.00000,2.49000,19.30746
Acadia,21.00000,80.20000,1.00000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Accomack,11.00000,74.20000,2.00000,20.80000,69.00000,51.50000,28.00000,0.60000,0.60000,0.20000,1.50000,9.00000,7.00000,2.25000,27.75811
Ada,71.00000,64.20000,1.00000,12.10000,92.50000,50.00000,1.30000,0.80000,2.60000,0.20000,2.60000,7.50000,5.90000,2.60000,176.65322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yuma,0.00000,74.40000,0.00000,17.20000,97.30000,49.90000,0.30000,0.90000,0.40000,0.20000,0.80000,21.60000,10.90000,2.61000,1.63608
Zapata,1.00000,84.40000,0.00000,11.20000,98.60000,49.60000,0.50000,0.40000,0.20000,0.00000,0.20000,93.50000,24.50000,3.17000,5.48326
Zavala,0.00000,83.20000,0.00000,12.50000,96.90000,50.20000,1.20000,0.90000,0.10000,0.10000,0.70000,92.90000,11.20000,3.15000,3.52352
Ziebach,0.00000,69.30000,0.00000,7.00000,24.30000,50.40000,0.40000,71.60000,0.30000,0.00000,3.40000,3.70000,0.90000,3.51000,0.54255


In [375]:
drop = list(dropped['variable'])
cases_US_curr = cases_US_curr.drop(drop,axis=1)

df1_styler = (vif.style
                  .set_table_attributes("style='display:inline'")
                  .set_caption('US VIF Values'))
df2_styler = (dropped.style
                  .set_table_attributes("style='display:inline'")
                  .set_caption('Variables removed for high multicolinearity'))

display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)



,variable,VIF
0,Cases_Delta,1.356432
1,Avg_Temp,5.076446
2,Protest_Count,2.103504
3,Perc.Black,1.519322
4,Perc.Native,1.359629
5,Perc.Asian,3.389680
6,Perc.Pac.Island,1.723153
7,Perc.Mixed,5.349729
8,Perc.His.Lat,3.759431
9,Perc.Foreign.Born,7.367302


In [382]:

x, y = cases_US_curr.iloc[:,:-1], cases_US_curr['Cases_2W']

# 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25,random_state=110) 

In [401]:
# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# # Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# # Method of selecting samples for training each tree
# bootstrap = [True, False]
# # Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'bootstrap': bootstrap}

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 11, num = 10)]
# Minimum number of samples required at each leaf node
min_samples_leaf = [int(x) for x in np.linspace(start = 1, stop = 5, num = 5)]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100,
                               cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

save_model(rf_random, "../models/random_forest_cv.sav")

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.0min finished


In [402]:
rf_random = load_model("../models/random_forest_cv.sav")
rf_model = rf_random.best_estimator_

In [413]:
results = pd.DataFrame(rf_random.cv_results_)
results.loc[results['rank_test_score'] > 1, 'color'] = 'blue'
results.loc[results['rank_test_score'] == 1, 'color'] = 'red'
best_n = results.loc[results['mean_test_score'] == results['mean_test_score'].max(), 'param_n_estimators'].values[0]

hover = HoverTool(tooltips=[
    ('Estimators', '@n_estimators'),
    ('Score', '@score{0.00000}'),
    ('Max Features', '@param_max_features'),
    ('Bootstrap', '@param_bootstrap')
    ])

p = figure(title = "CV of Random Forest Model", plot_height=500, plot_width=750,
           tools=[hover])

source = ColumnDataSource(data=dict(
    color=results['color'],
    n_estimators=results['param_n_estimators'],
    score=results['mean_test_score'],
    param_max_features=results['param_max_features'],
    param_bootstrap=results['param_bootstrap'],

))


p.circle('n_estimators', 'score',line_width=2, source=source)
vline = Span(location=best_n, dimension='height', line_color='red', line_dash='dashed',line_width=2)
p.renderers.extend([vline])

p.xaxis.axis_label = '# of Trees'
p.yaxis.axis_label = 'coefficient of determination (r²)'

output_file('../assets/img/Bokeh/rf_cv.html')

show(p)

<iframe src="assets/img/Bokeh/rf_cv.html"
    sandbox="allow-same-origin allow-scripts"
    width="100%"
    height="550"
    scrolling="no"
    seamless="seamless"
    frameborder="0">
</iframe>

In [410]:
results.loc[results['mean_test_score'] == results['mean_test_score'].max(), 'param_n_estimators'].values[0]

944

In [405]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    return pd.DataFrame({
        'Average RMSE': [sqrt(mean_squared_error(predictions, test_labels))],
        'Average MAE': [mean_absolute_error(predictions, test_labels)],
        'R2': [r2_score(predictions, test_labels)]
    })
evaluate(rf_model, X_test, y_test)

,Average RMSE,Average MAE,R2
0,1268.54348,325.19321,0.55932


In [406]:
importances = list(rf_model.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(list(cases_US.columns), importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1])
feature_importances = dict(feature_importances)


hover = HoverTool(tooltips=[
    ('Variable', '@Variables'),
    ('Importance', '@Importance{0.00}')
    ])

p = figure(y_range=list(feature_importances.keys()), plot_height=500, width=750, 
           title="Random Forest Variable Importance", tools=[hover])
source=ColumnDataSource(data=dict(
    Variables=list(feature_importances.keys()),
    Importance=list(feature_importances.values())
))

p.hbar(y='Variables', right='Importance',left=0, height=0.9, source=source)
p.yaxis.major_label_orientation = "horizontal"

p.xaxis.axis_label = 'Variable Importance'
p.yaxis.axis_label = 'Vasriable Name'
p.ygrid.grid_line_color = None

output_file('../assets/img/Bokeh/rf_imp.html')

show(p)

<iframe src="assets/img/Bokeh/rf_imp.html"
    sandbox="allow-same-origin allow-scripts"
    width="100%"
    height="550"
    scrolling="no"
    seamless="seamless"
    frameborder="0">
</iframe>

In [19]:
dot_data = StringIO()
export_graphviz(rf_model.estimators_[0], out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

InvocationException: GraphViz's executables not found

In [265]:
dates = cases_US.index.get_level_values(1)
sequence_length = max(dates)-min(dates)

def preprocess_data(data, label_name, process_vals):
    for col in process_vals:
        if data[col].dtype == 'int64' or data[col].dtype == 'float64':
            data[col] = data[col].pct_change() # normalizes variables
            data = data.replace([np.inf, -np.inf], np.nan).dropna()
            data[col] = preprocessing.scale(data[col].values)
    data.dropna(inplace=True)
    
    seq_data = []
    prev_days = deque(maxlen=sequence_length)
    for val in data.values:
        prev_days.append([n for n in val[:-1]])
        if len(prev_days) == sequence_length:
            seq_data.append([np.array(prev_days), val[-1]])
    
    random.shuffle(seq_data)
    X = []
    y = []
    
    for x_i, y_i in seq_data:
        X.append(x_i)
        y.append(y_i)
    
    return np.array(X), y

times = cases_US.index.get_level_values(1).values

last_5perc = times[-int(0.05*len(times))]

train_data = cases_US[(cases_US.index.get_level_values(1).values >= last_5perc)]
test_data = cases_US[(cases_US.index.get_level_values(1).values < last_5perc)]

X_train, y_train = preprocess_data(train_data, 'Cases_2W', ['Avg_Temp'])
X_test, y_test = preprocess_data(test_data, 'Cases_2W', ['Avg_Temp'])

In [262]:
X_train

array([[['minor/varies', 'R', '3', ..., '1.1106299212598423',
         '0.07716535433070869', '33.55708661417321'],
        ['minor/varies', 'R', '3', ..., '1.1106299212598423',
         '0.07716535433070869', '33.55708661417321'],
        ['minor/varies', 'R', '3', ..., '1.1106299212598423',
         '0.07716535433070869', '33.55708661417321'],
        ...,
        ['moderate/varies', 'D', '1', ..., '1.52421052631579',
         '0.05263157894736843', '3.7821052631578955'],
        ['moderate/varies', 'D', '2', ..., '1.52421052631579',
         '0.05263157894736843', '3.7821052631578955'],
        ['moderate/varies', 'D', '2', ..., '1.52421052631579',
         '0.05263157894736843', '3.7821052631578955']],

       [['moderate', 'D', '3', ..., '2.4', '0.08', '6.9'],
        ['minor', 'R', '2', ..., '0.9065217391304345',
         '0.08043478260869566', '4.397826086956521'],
        ['minor', 'R', '2', ..., '0.9065217391304345',
         '0.08043478260869566', '4.397826086956521'],
      

In [270]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

Epochs = 10
Batch_Size = 64

model = Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=[len(X_train)]),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dense(1)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

model.compile(loss='mse',
             optimizer=optimizer,
             metrics=['mae', 'mse'])

# model.predict(X_test)
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 64)                158272    
_________________________________________________________________
dense_22 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 65        
Total params: 162,497
Trainable params: 162,497
Non-trainable params: 0
_________________________________________________________________


In [268]:
X_test.shape

(4775, 160, 11)